In [1]:
import pandas as pd
import re

In [2]:
cta_train_gt = pd.read_csv('SOTAB CTA/sotab_v2_cta_training_set.csv')
cta_test_gt = pd.read_csv('SOTAB CTA/sample_test.csv')

In [ ]:
cta_train_gt

In [6]:
cta_test_gt

,table_name,column_index,label
0,Event_blugrottonj.com_September2020_CTA.json.gz,4,Place/name
1,Event_blugrottonj.com_September2020_CTA.json.gz,5,telephone
2,LocalBusiness_basellive.ch_September2020_CTA.j...,2,openingHours
3,LocalBusiness_worldcement.com_September2020_CT...,5,Country
4,Movie_321movies.org_September2020_CTA.json.gz,10,Rating
...,...,...,...
313,Product_babyartikelcheck.de_September2020_CTA....,0,Product/name
314,Product_divas-club.de_September2020_CTA.json.gz,0,Product/name
315,Product_divas-club.de_September2020_CTA.json.gz,1,Product/description
316,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber


In [8]:
cta_test_gt["class"] = cta_test_gt["table_name"].apply(lambda x: x.split("_")[0])

In [9]:
cta_test_gt

,table_name,column_index,label,class
0,Event_blugrottonj.com_September2020_CTA.json.gz,4,Place/name,Event
1,Event_blugrottonj.com_September2020_CTA.json.gz,5,telephone,Event
2,LocalBusiness_basellive.ch_September2020_CTA.j...,2,openingHours,LocalBusiness
3,LocalBusiness_worldcement.com_September2020_CT...,5,Country,LocalBusiness
4,Movie_321movies.org_September2020_CTA.json.gz,10,Rating,Movie
...,...,...,...,...
313,Product_babyartikelcheck.de_September2020_CTA....,0,Product/name,Product
314,Product_divas-club.de_September2020_CTA.json.gz,0,Product/name,Product
315,Product_divas-club.de_September2020_CTA.json.gz,1,Product/description,Product
316,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber,Person


In [4]:
gt = {'train':{}, 'test':{}}
for index, row in cta_train_gt.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in cta_test_gt.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]

In [5]:
def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

In [6]:
def get_all_table_columns(file_name, index):
    
    if file_name in cta_train_gt["table_name"].tolist():
        path = 'SOTAB CTA/Train/'+file_name
    else:
        path = 'SOTAB CTA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    
    for row in df.iloc[:, index].tolist():
        
        cleaned = " ".join(clean_text(row).split()[:20])
        
        if cleaned != "":
            cleaned_rows.append(cleaned)
    
    
    return " ".join(cleaned_rows[:5])

In [19]:
test_examples = []
for table in gt['test']:
    for column in gt['test'][table]:
        col_str = get_all_table_columns(table, column)
        test_examples.append([table, column, col_str, gt['test'][table][column], table.split("_")[0]])

In [ ]:
test_examples

In [7]:
train_examples = []
for table in gt['train']:
    for column in gt['train'][table]:
        col_str = get_all_table_columns(table, column)
        train_examples.append([table, column, col_str, gt['train'][table][column], table.split("_")[0]])

In [ ]:
train_examples

In [ ]:
len(train_examples)

In [16]:
import pickle

In [18]:
file_name='SOTAB CTA/sotabv2-cta-train-column.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()

In [17]:
file_name='SOTAB CTA/sotabv2-cta-sample-test-column.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()